In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import keras
from keras.models import load_model
# import os
#import seaborn as sns
#import category_encoders as ce

#from sklearn.preprocessing import MinMaxScaler, Imputer
#from xgboost import XGBClassifier
#from sklearn.metrics import accuracy_score
#from sklearn.metrics import confusion_matrix,classification_report

Using TensorFlow backend.


In [2]:
feature_XGB_no_fillnan = pd.read_csv('XGB_FE2_No_fillNaN.csv')

In [3]:
feature_XGB_no_fillnan.describe()

,SK_ID_CURR,TARGET
count,48744.000000,48744.000000
mean,277796.676350,0.209100
std,103169.547296,0.160662
min,100001.000000,0.017061
25%,188557.750000,0.084904
50%,277549.000000,0.156068
75%,367555.500000,0.291792
max,456250.000000,0.864964


In [9]:
feature_logistic_linear = pd.read_csv('log_reg_All_table_PCA_new_FE2.csv')

In [27]:
train_label = np.load('data/train_label.npy')
train_PCA = np.load('data/train_PCA.npy')
test_PCA = np.load('data/test_PCA.npy')
#train_eng = np.load('data/train_eng.npy')
#test_eng = np.load('data/test_eng.npy')
train_SK_ID_CURR = np.load('data/train_SK_ID_CURR.npy')
test_SK_ID_CURR = np.load('data/test_SK_ID_CURR.npy')
feature_XGB = pd.read_csv('data/train_XGB_FE2_No_fillNaN2.csv')
test_XGB = pd.read_csv('data/test_XGB_FE2_No_fillNaN2.csv')

In [3]:
#test_eng = pd.read_csv('data/Test_Eng2.csv')
#test_SK_ID_CURR = test_eng['SK_ID_CURR']
#del test_eng

In [4]:
#train_eng = pd.read_csv('data/Train_Eng2.csv')
#train_SK_ID_CURR = train_eng['SK_ID_CURR']
#del train_eng

In [5]:
#np.save('data/train_SK_ID_CURR', train_SK_ID_CURR)
#np.save('data/test_SK_ID_CURR', test_SK_ID_CURR)

In [5]:
# trainFeat = pd.DataFrame(train_PCA)
# testFeat = pd.DataFrame(test_PCA)
# trainFeat['XGB_no_fillnan'] = feature_XGB_no_fillnan['TARGET']
# testFeat['XGB_no_fillnan'] = feature_XGB_no_fillnan['TARGET']

In [7]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, Y_train, Y_test = train_test_split(train_PCA, train_label, test_size=0.2, random_state=42)

In [28]:
train_PCA = pd.DataFrame(train_PCA)

In [29]:
from sklearn import cross_validation
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(train_PCA, train_label, test_size=0.2, random_state=42, stratify=train_label)

In [45]:
feature_XGB.iloc[X_train.index]

,SK_ID_CURR,TARGET
181648,310536,0.168522
229245,365516,0.166022
122525,242055,0.410851
306311,454894,0.327058
300658,448321,0.527807
201033,333032,0.056296
86445,200322,0.282444
225267,360927,0.145468
42927,149698,0.160273
240820,378855,0.280072


In [46]:
del train_PCA

In [5]:
print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('Y_train', Y_train.shape)
print('Y_test', Y_test.shape)

X_train (246008, 994)
X_test (61503, 994)
Y_train (246008,)
Y_test (61503,)


In [9]:
print('There are', np.sum(Y_train), 'Trues out of 246008 = ', np.sum(Y_train)/246008.0, '%')

There are 19860 Trues out of 246008 =  0.08072908198107379 %


In [10]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train)
class_weights

array([0.5439093 , 6.19355488])

In [11]:
class_weight.compute_class_weight('balanced', np.unique(Y_test), Y_test)

array([0.54390852, 6.19365559])

# Time for Feed forward

In [204]:
from keras.layers import Dense, Input, Dropout
from keras.models import Model
from keras.optimizers import Adam

def get_feedforward_nn():    
    input1 = Input(shape=(994,))    
    x = Dense(512, activation='relu')(input1)
    x = Dropout(0.25)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(100, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer='adam', loss='binary_crossentropy')

    return model

In [205]:
from keras import backend as K
# This is called to clear the original model session in order to use TensorBoard
K.clear_session()

model_ff = get_feedforward_nn()
model_ff.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 994)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               509440    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
__________

In [206]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau#, TensorBoard

print('start training ff')

# Path to save model parameters
weight_path_model_ff ='model_ff_nn.h5'
# Path to write tensorboard
tensorboard_path_model_ff = 'Graphs/ff_nn'

callbacks_list_model_ff_nn = [
    #TensorBoard(log_dir=tensorboard_path_model_ff, histogram_freq=1, write_graph=True, write_grads=True),
    ModelCheckpoint(
            weight_path_model_ff,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        ),
    ReduceLROnPlateau(monitor='val_loss', factor=0.21, patience=2, min_lr=0.0001)
]

verbose = 1
epochs, batch_size = [10,3096]

model_ff.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=verbose, class_weight='auto',
                callbacks=callbacks_list_model_ff_nn, validation_data=(X_test, Y_test))

start training ff
Train on 246008 samples, validate on 61503 samples
Epoch 1/10
246008/246008 [==============================] - 2s 9us/step - loss: 0.2860 - val_loss: 0.2443

Epoch 00001: val_loss improved from inf to 0.24430, saving model to model_ff_nn.h5
Epoch 2/10
246008/246008 [==============================] - 2s 8us/step - loss: 0.2449 - val_loss: 0.2420

Epoch 00002: val_loss improved from 0.24430 to 0.24196, saving model to model_ff_nn.h5
Epoch 3/10
246008/246008 [==============================] - 2s 8us/step - loss: 0.2413 - val_loss: 0.2425

Epoch 00003: val_loss did not improve from 0.24196
Epoch 4/10
246008/246008 [==============================] - 2s 8us/step - loss: 0.2387 - val_loss: 0.2400

Epoch 00004: val_loss improved from 0.24196 to 0.24003, saving model to model_ff_nn.h5
Epoch 5/10
246008/246008 [==============================] - 2s 8us/step - loss: 0.2364 - val_loss: 0.2403

Epoch 00005: val_loss did not improve from 0.24003
Epoch 6/10
246008/246008 [===========

In [5]:
def evaluate(features, labels, model):
    """
    Evaluate model on validation data
    """
    return model.evaluate(x=features, y=labels, batch_size=batch_size, verbose=1, sample_weight=None, steps=None)

evaluate(X_test, Y_test, model_ff)

In [215]:
# prediction = regressor.predict(data[['X']])
# prediction = pd.DataFrame(predictions, columns=['predictions']).to_csv('prediction.csv')

# Submission dataframe
submit = pd.DataFrame(data=test_SK_ID_CURR, columns=['SK_ID_CURR'])
submit['TARGET'] = model_ff.predict(test_PCA)

# Save the submission to a csv file
submit.to_csv('FF_DO_PCA_AUTO_CALSS_WEIGHT.csv', index=False)

In [216]:
submit

,SK_ID_CURR,TARGET
0,100001,0.036999
1,100005,0.189854
2,100013,0.066468
3,100028,0.014913
4,100038,0.198488
5,100042,0.097491
6,100057,0.012948
7,100065,0.058493
8,100066,0.006691
9,100067,0.180698


In [210]:
K.clear_session()
model_ff = get_feedforward_nn()
model_ff.load_weights('model_ff_nn(0.77475).h5')

In [217]:
submit = pd.DataFrame(data=test_SK_ID_CURR, columns=['SK_ID_CURR'])
submit['TARGET'] = model_ff.predict(test_PCA)
submit.to_csv('data/test_FF.csv', index=False)

In [222]:
submit = pd.DataFrame(data=train_SK_ID_CURR, columns=['SK_ID_CURR'])
submit['TARGET'] = model_ff.predict(train_PCA)
submit.to_csv('data/train_FF.csv', index=False)

# Conv1D

#### preprocess data for cnn

In [49]:
X_train_cnn = X_train.values.reshape((-1,994,1))
X_test_cnn = X_test.values.reshape((-1,994,1))
X_train_feat_xgb = feature_XGB.iloc[X_train.index]['TARGET'].values
X_test_feat_xgb = feature_XGB.iloc[X_test.index]['TARGET'].values
del X_train
del X_test

In [71]:
from keras.layers import Dense, Input, Dropout, Conv1D, concatenate, Flatten
from keras.models import Model
from keras.optimizers import Adam
#keras.layers.Conv1D(filters, kernel_size, strides=1, padding='valid', data_format='channels_last', 
#dilation_rate=1, activation=None, use_bias=True, kernel_initializer='glorot_uniform', 
#bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, 
#kernel_constraint=None, bias_constraint=None)

def get_cnn():    
    input1 = Input(shape=(994, 1,), name='main_input')
    
    cv1 = Conv1D(filters=25, kernel_size=3, strides=1, activation='relu')(input1)
    cv1 = Dropout(0.25)(cv1)
    cv1 = Dense(60, activation='softmax')(cv1)
    cv1 = Conv1D(filters=25, kernel_size=3, strides=1, activation='relu')(cv1)
    cv1 = Dropout(0.25)(cv1)
    cv1 = Dense(10, activation='softmax')(cv1)
    cv1 = Flatten()(cv1)
    cv1 = Dense(200, activation='relu')(cv1)
    
    #cv2 = Conv1D(filters=120, kernel_size=3, strides=2, activation='relu')(input1)
    #cv2 = Dropout(0.25)(cv2)
    #cv2 = Dense(30, activation='softmax')(cv2)
    #cv2 = Flatten()(cv2)
    
    #cv3 = Conv1D(filters=100, kernel_size=3, strides=3, activation='relu')(input1)
    #cv3 = Dropout(0.25)(cv3)
    #cv3 = Dense(50, activation='softmax')(cv3)
    #cv3 = Flatten()(cv3)
    
    #cv4 = Conv1D(filters=70, kernel_size=3, strides=4, activation='relu')(input1)
    #cv4 = Dropout(0.25)(cv4)
    #cv4 = Dense(50, activation='softmax')(cv4)
    #cv4 = Flatten()(cv4)
    
    #cv5 = Conv1D(filters=50, kernel_size=3, strides=5, activation='relu')(input1)
    #cv5 = Dropout(0.25)(cv5)
    #cv5 = Dense(25, activation='softmax')(cv5)
    #cv5 = Flatten()(cv5)
    
    flatinput1 = Flatten()(input1)
    
    x = concatenate([flatinput1, cv1])
    
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(100, activation='relu')(x)
    xgb_pred = Input(shape=(1,), name='xgb_input')
    x = concatenate([xgb_pred, x])
    out = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[input1, xgb_pred], outputs=out)
    model.compile(optimizer='adam', loss='binary_crossentropy')

    return model

In [72]:
from keras import backend as K
# This is called to clear the original model session in order to use TensorBoard
K.clear_session()

model_cnn = get_cnn()
model_cnn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 994, 1)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 992, 25)      100         main_input[0][0]                 
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 992, 25)      0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 992, 60)      1560        dropout_1[0][0]                  
__________________________________________________________________________________________________
conv1d_2 (

In [73]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau#, TensorBoard

print('start training cnn')

# Path to save model parameters
weight_path_model_cnn ='model_cnn_ff_xgb.h5'
# Path to write tensorboard
tensorboard_path_model_cnn = 'Graphs/cnn'

callbacks_list_model_cnn = [
    #TensorBoard(log_dir=tensorboard_path_model_cnn, histogram_freq=1, write_graph=True, write_grads=True),
    ModelCheckpoint(
            weight_path_model_cnn,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        ),
    ReduceLROnPlateau(monitor='val_loss', factor=0.21, patience=2, min_lr=0.0001)
]

verbose = 1
epochs, batch_size = [9,3096]

model_cnn.fit([X_train_cnn, X_train_feat_xgb.values], Y_train, epochs=epochs, batch_size=batch_size, verbose=verbose, class_weight='auto',
                callbacks=callbacks_list_model_cnn, validation_data=([X_test_cnn, X_test_feat_xgb], Y_test))

start training cnn
Train on 246008 samples, validate on 61503 samples
Epoch 1/9
246008/246008 [==============================] - 39s 160us/step - loss: 0.2779 - val_loss: 0.2466

Epoch 00001: val_loss improved from inf to 0.24665, saving model to model_cnn_ff_xgb.h5
Epoch 2/9
246008/246008 [==============================] - 38s 154us/step - loss: 0.2482 - val_loss: 0.2453

Epoch 00002: val_loss improved from 0.24665 to 0.24528, saving model to model_cnn_ff_xgb.h5
Epoch 3/9
246008/246008 [==============================] - 38s 154us/step - loss: 0.2458 - val_loss: 0.2516

Epoch 00003: val_loss did not improve from 0.24528
Epoch 4/9
246008/246008 [==============================] - 38s 154us/step - loss: 0.2430 - val_loss: 0.2446

Epoch 00004: val_loss improved from 0.24528 to 0.24456, saving model to model_cnn_ff_xgb.h5
Epoch 5/9
246008/246008 [==============================] - 38s 154us/step - loss: 0.2407 - val_loss: 0.2425

Epoch 00005: val_loss improved from 0.24456 to 0.24251, saving

In [74]:
K.clear_session()
model_cnn = get_cnn()
model_cnn.load_weights(weight_path_model_cnn)

In [75]:
submit = pd.DataFrame(data=test_SK_ID_CURR, columns=['SK_ID_CURR'])
submit['TARGET'] = model_cnn.predict([test_PCA.reshape((-1,994,1)), test_XGB['TARGET']])
submit.to_csv('cnn_ff_xgb.csv', index=False)

In [76]:
submit

,SK_ID_CURR,TARGET
0,100001,0.041532
1,100005,0.201485
2,100013,0.058560
3,100028,0.015930
4,100038,0.185706
5,100042,0.135859
6,100057,0.018926
7,100065,0.053272
8,100066,0.006577
9,100067,0.251378


In [ ]:
submit = pd.DataFrame(data=train_SK_ID_CURR, columns=['SK_ID_CURR'])
submit['TARGET'] = model_cnn.predict(train_PCA.reshape((-1,994,1)))
submit.to_csv('data/train_cnn.csv', index=False)

In [ ]:
submit = pd.DataFrame(data=test_SK_ID_CURR, columns=['SK_ID_CURR'])
submit['TARGET'] = model_cnn.predict(test_PCA.reshape((-1,994,1)))
submit.to_csv('data/test_cnn.csv', index=False)